## Pythonノートブックにようこそ！

ディジャは私がいなくて寂しいですか？

###そして2週目、2日目へようこそ -  Langgraphの紹介！

In [1]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
import random


In [2]:
# いくつかの便利な定数

nouns = ["Cabbages", "Unicorns", "Toasters", "Penguins", "Bananas", "Zombies", "Rainbows", "Eels", "Pickles", "Muffins"]
adjectives = ["outrageous", "smelly", "pedantic", "existential", "moody", "sparkly", "untrustworthy", "sarcastic", "squishy", "haunted"]

In [ ]:
# 私たちのお気に入りの最初のステップ！ところで、乗組員は私たちのためにこれをやっていた。
load_dotenv(override=True)


In [ ]:
def shout(text: Annotated[str, "something to be shouted"]) -> str:
    print(text.upper())
    return text.upper()

shout("hello")

###「注釈」についての言葉

あなたはおそらくこれを知っています。タイプヒントは、Pythonの機能であり、何かの種類を指定できます。

`my_favorite_things：list`

しかし、あなたはこれを知らないかもしれません：

また、「注釈付き」と呼ばれるものを使用して、他の誰かが便利だと思う可能性のある情報を追加することもできます。

`my_favorite_things：annotated [list、"これらは私のいくつかです "]]`

Langgraphは、状態オブジェクトを定義するときにこの機能を使用する必要があります。

新しい価値で状態を更新するためにどの機能を呼び出すべきかを伝えることを望んでいます。

この関数は**還元剤**と呼ばれます。

Langgraphは、最も一般的なケースを処理する「add_messages」と呼ばれるデフォルトの還元剤を提供します。

そして、それは、なぜ国家がこのように見えるのかを説明することを願っています。


### ステップ1：状態オブジェクトを定義します

任意のPythonオブジェクトを使用できます。しかし、TypedDictまたはPydantic Basemodelを使用することが最も一般的です。

In [5]:

class State(BaseModel):
        
    messages: Annotated[list, add_messages]


### ステップ2：この状態クラスでグラフビルダーを起動します

In [6]:
graph_builder = StateGraph(State)

###ステップ3：ノードを作成します

ノードは任意のPython関数にすることができます。

前に設定した還元剤は自動的に呼び出され、この応答と以前の応答を組み合わせて


In [ ]:
def our_first_node(old_state: State) -> State:

    reply = f"{random.choice(nouns)} are {random.choice(adjectives)}"
    messages = [{"role": "assistant", "content": reply}]

    new_state = State(messages=messages)

    return new_state

graph_builder.add_node("first_node", our_first_node)

### ステップ4：エッジを作成します

In [ ]:
graph_builder.add_edge(START, "first_node")
graph_builder.add_edge("first_node", END)

### ステップ5：グラフをコンパイルします

In [9]:
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

### それでおしまい！ショータイム！

In [ ]:
def chat(user_input: str, history):
    message = {"role": "user", "content": user_input}
    messages = [message]
    state = State(messages=messages)
    result = graph.invoke(state)
    print(result)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

### しかし、なぜ私はあなたにそれを見せたのですか？

LanggraphがすべてPython関数に関するものであると主張するために -  LLMを巻き込む必要はありません!!

今、私たちは再び5つのステップを行いますが、1つのショットで：

In [12]:
# ステップ1：状態オブジェクトを定義します
class State(BaseModel):
    messages: Annotated[list, add_messages]


In [13]:
# ステップ2：この状態クラスでグラフビルダーを起動します
graph_builder = StateGraph(State)


In [ ]:
# ステップ3：ノードを作成します

llm = ChatOpenAI(model="gpt-4o-mini")

def chatbot_node(old_state: State) -> State:
    response = llm.invoke(old_state.messages)
    new_state = State(messages=[response])
    return new_state

graph_builder.add_node("chatbot", chatbot_node)

In [ ]:
# ステップ4：エッジを作成します
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
# ステップ5：グラフをコンパイルします
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

### それでおしまい！そして、これをしましょう：

In [ ]:
def chat(user_input: str, history):
    initial_state = State(messages=[{"role": "user", "content": user_input}])
    result = graph.invoke(initial_state)
    print(result)
    return result['messages'][-1].content


gr.ChatInterface(chat, type="messages").launch()